<a href="https://colab.research.google.com/github/TimHBSWFL/UCSD-ML-Capstone/blob/main/hyperparameter_tuning_yelp_reviews_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.4 MB/s eta 0:00:00


In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [15]:
import optuna
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer
from datasets import Dataset
from sklearn.model_selection import train_test_split

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
filename = "FL_Reviews_Edited" + ".csv"
directory = '/content/drive/My Drive/Capstone Data Collection/'

path = directory + filename

chunk_iterator = pd.read_csv(path, chunksize=10000)

chunks = []

for chunk in chunk_iterator:
  chunks.append(chunk)

df_reviews = pd.concat(chunks, ignore_index=True)
df_reviews.shape

(792133, 24)

In [17]:
df_sample = df_reviews.sample(frac=0.0025, random_state=42)
df_sample = df_sample[df_sample['stars_reviews'].isin([1,5])]
df_sample['binary_labels'] = df_sample['stars_reviews'].map({1: 0, 5: 1})
df_sample.shape

(1149, 25)

In [7]:
df_sample.columns

Index(['business_id', 'name', 'address', 'city_original', 'state',
       'postal_code', 'latitude', 'longitude', 'stars_business',
       'review_count', 'is_open', 'attributes', 'categories', 'hours',
       'review_id', 'user_id', 'stars_reviews', 'useful', 'funny', 'cool',
       'text', 'date', 'zip_code', 'city_updated', 'binary_labels'],
      dtype='object')

In [18]:
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df_sample['text'].tolist(), df_sample['binary_labels'].tolist(), test_size=0.2, random_state=42
)

train_dataset = Dataset.from_dict({'text': train_texts, 'label': train_labels})
test_dataset = Dataset.from_dict({'text': test_texts, 'label': test_labels})

In [19]:
df_sample['stars_reviews'].value_counts()

,count
stars_reviews,
5,930
1,219


In [10]:
distilbert_model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(distilbert_model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [20]:
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True, max_length=512)

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

Map:   0%|          | 0/919 [00:00<?, ? examples/s]

Map:   0%|          | 0/230 [00:00<?, ? examples/s]

In [23]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=-1)
    accuracy = accuracy_score(labels, predictions)
    return {
        'eval_accuracy': accuracy
    }


def objective(trial):
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 5e-5)
    batch_size = trial.suggest_categorical('batch_size', [8, 16, 32, 64])
    num_epochs = trial.suggest_int('num_epochs', 2, 5)

    model = AutoModelForSequenceClassification.from_pretrained(distilbert_model_name, num_labels=2)

    training_args = TrainingArguments(
        output_dir="./results",
        evaluation_strategy="epoch",
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,
        weight_decay=0.01,
        logging_dir="./logs",
        logging_steps=10,
        report_to="none"
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics
    )

    trainer.train()

    eval_results = trainer.evaluate()
    accuracy = eval_results["eval_accuracy"]

    return accuracy


In [24]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=5)

print("Best hyperparameters found:", study.best_params)

[I 2024-11-13 04:29:01,015] A new study created in memory with name: no-name-ebf43878-b815-4d64-be52-5806ea09fcd1
<ipython-input-23-0f3a95bfb5b8>:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 5e-5)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.092400,0.065689,0.978261
2,0.052300,0.045413,0.991304
3,0.019300,0.044549,0.991304
4,0.021600,0.044781,0.991304
5,0.011700,0.044791,0.991304


[I 2024-11-13 04:32:35,462] Trial 0 finished with value: 0.991304347826087 and parameters: {'learning_rate': 1.0080248949472861e-05, 'batch_size': 64, 'num_epochs': 5}. Best is trial 0 with value: 0.991304347826087.
<ipython-input-23-0f3a95bfb5b8>:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 5e-5)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.093900,0.061647,0.982609
2,0.047700,0.045319,0.991304
3,0.019200,0.044712,0.991304


[I 2024-11-13 04:34:49,449] Trial 1 finished with value: 0.991304347826087 and parameters: {'learning_rate': 1.3111640586391319e-05, 'batch_size': 64, 'num_epochs': 3}. Best is trial 0 with value: 0.991304347826087.
<ipython-input-23-0f3a95bfb5b8>:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 5e-5)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


...

Epoch,Training Loss,Validation Loss,Accuracy
1,0.177500,0.049767,0.991304
2,0.001900,0.048151,0.991304
3,0.000200,0.048826,0.991304


[I 2024-11-13 04:41:04,980] Trial 3 finished with value: 0.991304347826087 and parameters: {'learning_rate': 3.103292374641509e-05, 'batch_size': 8, 'num_epochs': 3}. Best is trial 0 with value: 0.991304347826087.
<ipython-input-23-0f3a95bfb5b8>:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 5e-5)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.094200,0.046153,0.991304
2,0.038800,0.041561,0.991304


[I 2024-11-13 04:42:44,994] Trial 4 finished with value: 0.991304347826087 and parameters: {'learning_rate': 2.299882008748234e-05, 'batch_size': 64, 'num_epochs': 2}. Best is trial 0 with value: 0.991304347826087.


Best hyperparameters found: {'learning_rate': 1.0080248949472861e-05, 'batch_size': 64, 'num_epochs': 5}


In [25]:
print("Best hyperparameters found:", study.best_params)

Best hyperparameters found: {'learning_rate': 1.0080248949472861e-05, 'batch_size': 64, 'num_epochs': 5}


In [33]:
df_sample = df_reviews.sample(frac=0.0075, random_state=42)
df_sample = df_sample[df_sample['stars_reviews'].isin([1,5])]
df_sample['binary_labels'] = df_sample['stars_reviews'].map({1: 0, 5: 1})
df_sample.shape

(3483, 25)

**Use best hyperparameters with "distilbert-base-uncased-finetuned-sst-2-english" model.**

In [34]:
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df_sample['text'].tolist(), df_sample['binary_labels'].tolist(), test_size=0.2, random_state=42
)

train_dataset = Dataset.from_dict({'text': train_texts, 'label': train_labels})
test_dataset = Dataset.from_dict({'text': test_texts, 'label': test_labels})

def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True, max_length=512)

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

Map:   0%|          | 0/2786 [00:00<?, ? examples/s]

Map:   0%|          | 0/697 [00:00<?, ? examples/s]

In [35]:
best_trial = study.best_trial
best_learning_rate = best_trial.params['learning_rate']
best_batch_size = best_trial.params['batch_size']
best_num_epochs = best_trial.params['num_epochs']

print(f"Best hyperparameters: Learning Rate: {best_learning_rate}, Batch Size: {best_batch_size}, Epochs: {best_num_epochs}")

model = AutoModelForSequenceClassification.from_pretrained(distilbert_model_name, num_labels=2)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=best_learning_rate,
    per_device_train_batch_size=best_batch_size,
    per_device_eval_batch_size=best_batch_size,
    num_train_epochs=best_num_epochs,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none"
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

eval_results = trainer.evaluate()

final_accuracy = eval_results["eval_accuracy"]
print(f"Final evaluation accuracy: {final_accuracy}")


Best hyperparameters: Learning Rate: 1.0080248949472861e-05, Batch Size: 64, Epochs: 5


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.047200,0.044957,0.989957
2,0.016200,0.043591,0.988522
3,0.016500,0.049377,0.988522
4,0.005400,0.048471,0.988522
5,0.001500,0.049587,0.989957


Final evaluation accuracy: 0.9899569583931134
